In [1]:
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch

import torch_geometric.transforms as T
from torch_geometric.nn import GatedGraphConv
from torch_geometric.loader import DataLoader

from dataset_processing import RNADataset


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print(device)

dataset = RNADataset(root="./data/")
dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)


In [3]:
def del_nucl_conn(y):
    main_diag = torch.zeros([1, y.size(0)])
    diag1 = torch.ones([1, y.size(0) - 1])

    return y * (torch.diag_embed(main_diag).to(device) + 
                torch.diag_embed(diag1, offset=1).to(device) + 
            torch.diag_embed(diag1, offset=-1).to(device) - 1)[0] * (-1)

In [4]:
def del_main_diag(y):
    main_diag = torch.ones([1, y.size(0)])
    return y * (torch.diag_embed(main_diag).to(device) - 1)[0] * (-1)

In [5]:
model(train_data[0].x, train_data[0].edge_index, train_data[0].edge_weight).size()

NameError: name 'model' is not defined

In [40]:
a = torch.rand(4, 4)
print(a)
# print(del_nucl_conn(a))
b = del_nucl_conn(a.to(device))
print(b)
# argm(b)
c = argm(b)
print(c)
# argm(b)
print(del_main_diag(c))

tensor([[0.7392, 0.6348, 0.3488, 0.0417],
        [0.9260, 0.6983, 0.8660, 0.3560],
        [0.8640, 0.0759, 0.8559, 0.6654],
        [0.4644, 0.8148, 0.4016, 0.1117]])
tensor([[0.7392, -0.0000, 0.3488, 0.0417],
        [-0.0000, 0.6983, -0.0000, 0.3560],
        [0.8640, -0.0000, 0.8559, -0.0000],
        [0.4644, 0.8148, -0.0000, 0.1117]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 0., 1., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]])
tensor([[-0., 0., 1., 0.],
        [0., -0., 0., 1.],
        [1., 0., -0., 0.],
        [0., 1., 0., -0.]])


In [6]:
def argm(y):
    max_ind = torch.argmax(y, dim=1)
    y1 = torch.zeros_like(y)
    k = 0
    print(y1)
    for i in max_ind:
        y1[k][i] = 1.
        y1[i][k] = 1.
        k += 1
#     print(y1)
    return y1.to(device)


In [7]:
def precision(y_pred, y_true):
    y_pred = del_nucl_conn(y_pred)
#     y_pred[(y_pred > 0.5)] = 1
#     y_pred[(y_pred <= 0.5)] = 0 
    y_pred = argm(y_pred)
    y_pred = del_main_diag(y_pred)
    
    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):
    y_pred = del_nucl_conn(y_pred)
#     y_pred[(y_pred > 0.5)] = 1
#     y_pred[(y_pred <= 0.5)] = 0 
    y_pred = argm(y_pred)
    y_pred = del_main_diag(y_pred)
    
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f_loss(y_pred, y_true):
    y_pred = del_nucl_conn(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    k = 1
    k1 = 1 - torch.abs(precision - recall)

#     k2 = 1 - torch.abs(K.mean(precision) - K.mean(recall))
    #calculate upgraded f1 score
    f1 = ((1 + k * k) * precision * recall) / (k * k * precision + recall + epsilon)
#     tw = K.sum(K.cast(y_true * y_pred, ’float32’), axis=[1, 2, 3])
#     fw = K.sum(K.cast((1 - y_true) * y_pred, ’float32’), axis=[1, 2, 3])
#     fb = K.sum(K.cast(y_true * (1 - y_pred), ’float32’), axis=[1, 2, 3])
    return 1 - f1


In [8]:
class GatedGCNModel(torch.nn.Module):
    def __init__(self, num_layers, out_channels):
        super(GatedGCNModel, self).__init__()
        self.ggcn = GatedGraphConv(out_channels, num_layers)
#         self.gcn = GCN(out_channels, hidden_channels, 2, 64, dropout=dropout)

    def forward(self, x, edge_index, edge_weight):
        x = self.ggcn(x, edge_index)
#         x = self.gcn(x.to(torch.float), edge_index)
        prob_adj = (x @ x.t()).sigmoid()

#         prob_adj = (x @ x.t())

#         return (prob_adj > 0).nonzero(as_tuple=False).t()
        return prob_adj


In [9]:
hidden_channels = 128
out_channels = 200
num_layers = 6
num_features = dataset.num_features


dr = 0.7
lr = 0.00002
epochs = 1500
beta = 1


model = GatedGCNModel(num_layers, out_channels)
# model = torch.load("./models1/GatedGCN_6_4e-05_Adam_200_2_60.pt")

model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.99))
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
# optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# criterion = torch.nn.BCELoss()
# criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.KLDivLoss()

# def RMSELoss(y_pred, y_true):
#     return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

criterion = f_loss

cpu


In [11]:
run_name = "GatedGCN_argm" + str(num_layers) + "_" + str(lr) + "_" + "Adam_" + str(out_channels) + "_" + str(beta)


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "GatedGCN",
    "epochs": epochs,
    "optimizer": "Adam",
    "out_channels": out_channels,
    "loss": "f_loss",
    "beta": beta,
    "train:val:test": "655:218:218"
    },
    name=run_name
)

epsilon = 1e-10

wandb: Currently logged in as: chi-vinny0702. Use `wandb login --relogin` to force relogin


In [12]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
        for g in tqdm(train_data, ncols=100):
            print(1)
            g.to(device)
            optimizer.zero_grad()

            out = model(g.x, g.edge_index, g.edge_weight)
            y_true = g.adj_mat
            loss = criterion(out, y_true.to(torch.float32))            
            
            loss.backward()
            optimizer.step()
            
#             out = out.sigmoid()
            
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
            for g in tqdm(val_data, ncols=100):
                g.to(device)
                out = model(g.x, g.edge_index, g.edge_weight)
        
                y_true = g.adj_mat
                loss = criterion(out, y_true.to(torch.float32))
                                
                val_loss.append(loss.item())
                
#                 out = out.sigmoid()
                
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [ ]:
train()
torch.save(model, "./models1/" + run_name + "_" + str(epochs) + ".pt")

  0%|                                                                       | 0/655 [00:00<?, ?it/s]

1


In [ ]:
run_name = "GatedGCN_" + str(num_layers) + "_" + "140" + "_" + str(lr) + "_" + "Adam_" + str(out_channels)


torch.save(model, "./models/" + run_name + ".pt")